In [1]:
import warnings
warnings.filterwarnings("ignore")

from xai_agg import *

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.ensemble import RandomForestClassifier

import pandas as pd
import numpy as np

import dill

2025-01-22 23:38:38.742630: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-01-22 23:38:39.004299: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
raw_data = pd.read_csv("../data/Student_Depression_Dataset.csv")
display(raw_data)
display(raw_data["Profession"].value_counts())

,id,Gender,Age,City,Profession,Academic Pressure,Work Pressure,CGPA,Study Satisfaction,Job Satisfaction,Sleep Duration,Dietary Habits,Degree,Have you ever had suicidal thoughts ?,Work/Study Hours,Financial Stress,Family History of Mental Illness,Depression
0,2,Male,33.0,Visakhapatnam,Student,5.0,0.0,8.97,2.0,0.0,5-6 hours,Healthy,B.Pharm,Yes,3.0,1.0,No,1
1,8,Female,24.0,Bangalore,Student,2.0,0.0,5.90,5.0,0.0,5-6 hours,Moderate,BSc,No,3.0,2.0,Yes,0
2,26,Male,31.0,Srinagar,Student,3.0,0.0,7.03,5.0,0.0,Less than 5 hours,Healthy,BA,No,9.0,1.0,Yes,0
3,30,Female,28.0,Varanasi,Student,3.0,0.0,5.59,2.0,0.0,7-8 hours,Moderate,BCA,Yes,4.0,5.0,Yes,1
4,32,Female,25.0,Jaipur,Student,4.0,0.0,8.13,3.0,0.0,5-6 hours,Moderate,M.Tech,Yes,1.0,1.0,No,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27896,140685,Female,27.0,Surat,Student,5.0,0.0,5.75,5.0,0.0,5-6 hours,Unhealthy,Class 12,Yes,7.0,1.0,Yes,0
27897,140686,Male,27.0,Ludhiana,Student,2.0,0.0,9.40,3.0,0.0,Less than 5 hours,Healthy,MSc,No,0.0,3.0,Yes,0
27898,140689,Male,31.0,Faridabad,Student,3.0,0.0,6.61,4.0,0.0,5-6 hours,Unhealthy,MD,No,12.0,2.0,No,0
27899,140690,Female,18.0,Ludhiana,Student,5.0,0.0,6.88,2.0,0.0,Less than 5 hours,Healthy,Class 12,Yes,10.0,5.0,No,1


Profession
Student                   27870
Architect                     8
Teacher                       6
Digital Marketer              3
Content Writer                2
Chef                          2
Doctor                        2
Pharmacist                    2
Civil Engineer                1
UX/UI Designer                1
Educational Consultant        1
Manager                       1
Lawyer                        1
Entrepreneur                  1
Name: count, dtype: int64

In [3]:
preprocessed_data = raw_data.drop(columns=["id", "City", "Profession", "Degree"])

# Treat column names
preprocessed_data.rename(columns={
    'Have you ever had suicidal thoughts ?': 'Suicidal',
    "Family History of Mental Illness": "Family_History",
    "Gender": "GenderMale"
}, inplace=True)
preprocessed_data.columns = preprocessed_data.columns.str.replace(' ', '_')
preprocessed_data.columns = preprocessed_data.columns.str.replace('/', '_')

# Encoding categorical variables
categorical_features = []
preprocessed_data["GenderMale"] = preprocessed_data["GenderMale"].map({"Male": 1, "Female": 0})
preprocessed_data["Dietary_Habits"] = preprocessed_data["Dietary_Habits"].map({"Others": 0, "Unhealthy": 1, "Moderate": 2, "Healthy": 3})
preprocessed_data["Suicidal"] = preprocessed_data["Suicidal"].map({"Yes": 1, "No": 0})
preprocessed_data["Family_History"] = preprocessed_data["Family_History"].map({"Yes": 1, "No": 0})
preprocessed_data["Sleep_Duration"] = preprocessed_data["Sleep_Duration"].map({"Others": 0, "Less than 5 hours": 1, "5-6 hours": 2, "7-8 hours": 3, "More than 8 hours": 4})

preprocessed_data, _ = train_test_split(preprocessed_data, test_size=0.875, stratify=preprocessed_data['Depression'], random_state=42)
preprocessed_data.dropna(inplace=True)

display(preprocessed_data)

,GenderMale,Age,Academic_Pressure,Work_Pressure,CGPA,Study_Satisfaction,Job_Satisfaction,Sleep_Duration,Dietary_Habits,Suicidal,Work_Study_Hours,Financial_Stress,Family_History,Depression
22551,0,33.0,3.0,0.0,5.86,5.0,0.0,2,1,1,8.0,1.0,1,0
15512,1,32.0,3.0,0.0,6.16,4.0,0.0,4,2,0,0.0,3.0,0,0
21944,1,19.0,3.0,0.0,7.80,4.0,0.0,4,2,0,2.0,1.0,0,0
15462,1,32.0,4.0,0.0,6.89,3.0,0.0,2,2,1,5.0,1.0,0,1
21760,0,33.0,2.0,0.0,8.17,2.0,0.0,3,3,0,6.0,4.0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10927,1,22.0,5.0,0.0,5.27,1.0,0.0,4,1,0,9.0,5.0,0,1
12611,0,23.0,5.0,0.0,6.27,1.0,0.0,1,1,0,1.0,4.0,1,1
14848,1,21.0,1.0,0.0,8.59,1.0,0.0,3,2,0,3.0,2.0,1,0
13476,0,29.0,1.0,0.0,9.71,3.0,0.0,4,2,0,6.0,4.0,0,0


In [4]:
y = preprocessed_data["Depression"]
X = preprocessed_data.drop(columns=["Depression"])

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [5]:
clf = RandomForestClassifier(random_state=42)
clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)

print(f"Accuracy: {accuracy_score(y_test, y_pred)}")
print(f"ROC AUC: {roc_auc_score(y_test, y_pred)}")

Accuracy: 0.8151862464183381
ROC AUC: 0.8017150225660865


# Experiments
# RAE-T vs. RAE-E | 10 samples
### Execution

In [6]:
results, metadata = evaluate_aggregate_explainer(
    clf, X_train, X_test, categorical_features,
    metrics_sets=[['nrc', 'sensitivity_spearman', 'rb_faithfulness_corr']],
    mcdm_algs=[pymcdm.methods.TOPSIS(), pymcdm.methods.EDAS()],
    n_instances=10
)

metadata["description"] = "RAE-T vs RAE-S, 10 samples"

with open('pickles/student_depression/RAE-T_vs_RAE-S_10-allrank.pkl', 'wb') as f:
    dill.dump(ExperimentRun(metadata, results), f)

Selected indexes: [ 2374  5094 14399   354  3879 14730  9360 17960  7977 19307]
Epoch 1/500
70/70 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 1.1097 - val_loss: 1.0697
Epoch 2/500
70/70 ━━━━━━━━━━━━━━━━━━━━ 0s 591us/step - loss: 1.0333 - val_loss: 1.0140
Epoch 3/500
70/70 ━━━━━━━━━━━━━━━━━━━━ 0s 667us/step - loss: 0.9945 - val_loss: 0.9659
Epoch 4/500
70/70 ━━━━━━━━━━━━━━━━━━━━ 0s 681us/step - loss: 0.9447 - val_loss: 0.9231
Epoch 5/500
70/70 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.9064 - val_loss: 0.8854
Epoch 6/500
70/70 ━━━━━━━━━━━━━━━━━━━━ 0s 815us/step - loss: 0.8609 - val_loss: 0.8522
Epoch 7/500
70/70 ━━━━━━━━━━━━━━━━━━━━ 0s 884us/step - loss: 0.8304 - val_loss: 0.8239
Epoch 8/500
70/70 ━━━━━━━━━━━━━━━━━━━━ 0s 724us/step - loss: 0.8117 - val_loss: 0.8001
Epoch 9/500
70/70 ━━━━━━━━━━━━━━━━━━━━ 0s 829us/step - loss: 0.7891 - val_loss: 0.7811
Epoch 10/500
70/70 ━━━━━━━━━━━━━━━━━━━━ 0s 781us/step - loss: 0.7844 - val_loss: 0.7660
Epoch 11/500
70/70 ━━━━━━━━━━━━━━━━━━━━ 0s 668us/step

### Analysis

In [7]:
with open('pickles/student_depression/RAE-T_vs_RAE-S_10-allrank.pkl', 'rb') as f:
    exp = dill.load(f)

In [8]:
methods = ["RAE-T", "RAE-E"]
for i, method in enumerate(methods):
    print(f"{method}:\n")
    display(exp.results[i])
    wca = count_worst_case_avoidances(exp.results[i], [False, True, True], 1)
    print(f"Worst case avoidances:\n\t- for all metrics: {wca[0]}\n\t- for 2/3 metrics: {wca[1]}")
    print("AVG:")
    display(get_expconfig_mean_results(exp, i))
    print("\n")
    print("Avg rank:")
    display(get_average_metric_rank(exp.results[i], [False, True, True]))

RAE-T:



[                              nrc  sensitivity_spearman  rb_faithfulness_corr
 LimeWrapper             21.211467              0.983516              0.880819
 ShapTabularTreeWrapper  20.741480              0.994490              0.920241
 AnchorWrapper           17.559590              1.000000              0.747207
 AggregateExplainer      17.886925              0.976374              0.743415,
                               nrc  sensitivity_spearman  rb_faithfulness_corr
 LimeWrapper             20.629599              0.956044              0.220449
 ShapTabularTreeWrapper  20.192111              0.965840              0.232545
 AnchorWrapper           16.960760              0.835722              0.231393
 AggregateExplainer      18.905932              0.826374              0.290000,
                               nrc  sensitivity_spearman  rb_faithfulness_corr
 LimeWrapper             21.688702              0.974725              0.403972
 ShapTabularTreeWrapper  21.454425              0.

Worst case avoidances:
	- for all metrics: 5
	- for 2/3 metrics: 9
AVG:


,nrc,sensitivity_spearman,rb_faithfulness_corr
AggregateExplainer,18.214384,0.946429,0.559827
AnchorWrapper,17.197114,0.870992,0.567401
LimeWrapper,20.443514,0.960989,0.441468
ShapTabularTreeWrapper,20.970760,0.990193,0.556680




Avg rank:


...

[                              nrc  sensitivity_spearman  rb_faithfulness_corr
 LimeWrapper             19.249676              0.939560              0.879739
 ShapTabularTreeWrapper  20.741480              0.998347              0.887028
 AnchorWrapper           17.559590              0.970131              0.691835
 AggregateExplainer      17.762886              0.935165              0.741337,
                               nrc  sensitivity_spearman  rb_faithfulness_corr
 LimeWrapper             21.211467              0.973626              0.178879
 ShapTabularTreeWrapper  20.192111              0.972452              0.302388
 AnchorWrapper           18.161386              0.843967              0.013994
 AggregateExplainer      18.745550              0.861538              0.276032,
                               nrc  sensitivity_spearman  rb_faithfulness_corr
 LimeWrapper             22.938190              0.966484              0.208088
 ShapTabularTreeWrapper  21.454425              0.

Worst case avoidances:
	- for all metrics: 7
	- for 2/3 metrics: 9
AVG:


,nrc,sensitivity_spearman,rb_faithfulness_corr
AggregateExplainer,18.000209,0.921154,0.575966
AnchorWrapper,18.385744,0.845828,0.477188
LimeWrapper,20.802944,0.970220,0.419113
ShapTabularTreeWrapper,20.970760,0.989917,0.565473




Avg rank:


,nrc,sensitivity_spearman,rb_faithfulness_corr
AggregateExplainer,1.8,3.3,2.1
AnchorWrapper,1.4,3.5,2.6
LimeWrapper,3.3,2.1,3.1
ShapTabularTreeWrapper,3.5,1.1,2.2
